# Tube Test Elo rating Calculation

## Importing other Python Libraries/Modules

In [1]:
import re
import os
import sys
import string
import glob
import ast
from collections import Counter
from collections import defaultdict
import warnings

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import git
# Getting the path of the root directory so that we can import repo specific functions
git_repo_object = git.Repo('.', search_parent_directories=True)
git_repo_directory = git_repo_object.working_tree_dir

In [4]:
# Setting path so that we can import functions
sys.path.append(os.path.join(git_repo_directory, "src"))

In [5]:
from elorating import calculation
from elorating import dataframe

In [6]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

# Summary

- Experiment
    - Observing competitve behavior between two mice inside a tube. Mice are categorized as winner and loser based on if they pushed the other mouse out of the tube or were pushed.
- Data
    - Excel spreadsheet of recorded tube test results. The relevant columns are those of the "Date" of the recording, the "winner" of the interaction, and the "loser" of the interaction. Each row will be for one interaction between two mice.
    - There is a cage for each sheet of the spreadsheet
    - For each recording session, we will assume that the date will be specified for that first row. We will use this to seperate all the rows into seperate sessions.
- Purpose of this Jupyter Notebook
    - To calculate the Elo rating of each mice after each interaction. The mice start off with an Elo rating of 1000. Elo ratings are calculated with the formula from here: https://www.omnicalculator.com/sports/elo . Then we will plot the change of Elo rating across all interactions. With the number of interactions on the X-Axis and the current Elo rating on the Y. There will be a line for mice.

## Name of protocol for naming

- This name will be used to name files and title plots. Please change if you are using a different protocol or adding more details
    - **NOTE**: This should be changed based on the name the protocol

In [7]:
protocol_name = "tube_test"

In [8]:
prefix_name = "tt"

In [9]:
cohort_name = ""

In [10]:
if not cohort_name:
    cohort_name = input("""Type out the name of the cohort you are using. 
                        Make sure that this is typed as one word with individual words separated by underscore. i.e. pilot_3""")

Type out the name of the cohort you are using. 
                        Make sure that this is typed as one word with individual words separated by underscore. i.e. pilot_3pilot_3


## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [11]:
input_folder = os.path.join(".", "data")

In [12]:
# Accomodates for both capitalization of the file names
raw_data_file_keyword = '*[tT]ube*'

# NOTE: If there is a set excel file used for this notebook, make the cell below into a code cell and put the path in the "" (quotation marks)

In [13]:
raw_data_file_path = ""

- Asking the user what the path to the recording files are, with the option of using wildcards

In [14]:
if not raw_data_file_path:
    raw_data_glob_pattern = input("""Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    """)

Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    


- By default, this will search for files that are in the `./data` folder (in the folder that this notebook is in) that have key word you specified in the file name

In [15]:
if not raw_data_file_path:
    # Using the user inputted path/pattern
    if raw_data_glob_pattern.strip():
        # Getting a list of all matching files
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Checking if there were any files that matched
        if not raw_data_files_list:
            raise ValueError("No files were found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))

    # Using the default pattern
    else:
        raw_data_glob_pattern = os.path.join(input_folder + "**", raw_data_file_keyword)
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Searching for matching files with recursion
        if not raw_data_files_list:
            raise ValueError("No files were found in {} that had the keyword {} in it".format(input_folder, raw_data_file_keyword))


- Checking to see if only one file is specified

In [16]:
if not raw_data_file_path:
    if len(raw_data_files_list) >= 2:
        raise ValueError("More than one file was found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))
    else:
        # Using the first(and only file) as the file path
        raw_data_file_path = raw_data_files_list[0]

In [17]:
raw_data_file_path

'.\\data\\pilot_3_tube_test.xlsx'

## Getting a list of all the sheets

In [18]:
# Getting the sheet names for the excel file
xls = pd.ExcelFile(raw_data_file_path)
raw_data_sheet_names = xls.sheet_names

In [19]:
raw_data_sheet_names

['CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6']

# NOTE: If there is a set sheet names used for this notebook, edit the cell below with the name of the sheets each in quotation marks seperated by commas in the brackets
   - i.e. `['CAGE 1', 'CAGE 2', 'CAGE 3', 'CAGE 4']`

In [20]:
inputted_sheet_names_list = []

- Asking the user what sheets they want to use

In [21]:
if not inputted_sheet_names_list:
    inputted_sheet_names_string = input("""Type out the name of the sheets that you want to be processed. 
    Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

    The available sheets are: {}

    Alternatively, you can use the index of the list of names above. 
    Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
    i.e. 1, 2

    NOTE: If left blank, all sheets will be used
    """.format(raw_data_sheet_names))

Type out the name of the sheets that you want to be processed. 
    Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

    The available sheets are: ['CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6']

    Alternatively, you can use the index of the list of names above. 
    Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
    i.e. 1, 2

    NOTE: If left blank, all sheets will be used
    


In [22]:
if not inputted_sheet_names_list:
    # Making a list out of the string of inputted sheet names
    if inputted_sheet_names_string:
        inputted_sheet_names_string = "[" + inputted_sheet_names_string + "]"
        # Turning the string into a list
        inputted_sheet_names_list = ast.literal_eval(inputted_sheet_names_string)
    # Using all the sheet names if no sheet name is specified
    else:
        inputted_sheet_names_list =  raw_data_sheet_names

In [23]:
inputted_sheet_names_list

['CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6']

- Converting all the numbers into the sheet name that the index of the number corresponds to

In [24]:
if inputted_sheet_names_list:
    for index, sheet in enumerate(inputted_sheet_names_list):
        # Checking if the sheet name was a number
        if isinstance(sheet, int) and str(sheet).isdigit():
            inputted_sheet_names_list[index] =  raw_data_sheet_names[sheet]

In [25]:
inputted_sheet_names_list

['CAGE1', 'CAGE2', 'CAGE3', 'CAGE4', 'CAGE5', 'CAGE6']

- Checking to see if all the sheets are in the excel file

In [26]:
if not set(inputted_sheet_names_list).issubset(raw_data_sheet_names):
    # Getting all the sheets that were not in the original spreadsheet
    not_included_sheet_names = set(inputted_sheet_names_list) - set(raw_data_sheet_names)
    raise ValueError("All the listed sheet names are not in {}".format(not_included_sheet_names))

## Finding the row for the header

- Headers are the row in a spreadsheet that has all the column names. Sometimes spreadsheets don't use the first row as the row with the column names. So, the row for the header will be asked or assumed to be the first row

# NOTE: If there is a set row that has the column names, then enter the row number in the quotation marks

In [27]:
# Making the row number a string, so that "0" is treated as a True value
all_header_row = ""
if not all_header_row:
    all_header_row = False
else:
    all_header_row = int(all_header_row)


In [28]:
sheet_name_to_everything = defaultdict(dict)
for sheet in inputted_sheet_names_list:
    if all_header_row is False:

        print("\nCurrent Sheet Name: {}".format(sheet))    
        per_sheet_dataframe = pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0)

        # Showing the columns that are chosen with the header being the 0th row
        print("Columns Names: {}".format(per_sheet_dataframe.columns))
        # Show the dataframe that would be created with the header being the 0th row
        print("First few rows of this dataframe:")
        print(pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0).head())

        # Allowing the user the choose the row number for the header
        header_row = input("""Type the row number to be used as the header
        (AKA the row with the column name that you want to use.)
        If you want to keep the column names that were displayed, type 0.
        If you want to use a different row, then type the corresponding number. 

        The rows displayed in this cell are dataframes created from Pandas. 
        To use the row with the 0 index for column names, type 1. 
        For the row with the 1 index, it will be 2 and so on. i.e. 2

        If you are looking at the original spread sheet, remember that Python is zero indexed. 
        So the first row will be 0, second will be 1, and so on. 
        i.e. 1

        NOTE: If left blank, the original row that was used will be used.
        """).strip()

        if header_row == "":
            header_row = 0
        else:
            header_row = int(header_row)
    
    # Checking if any of the column names are from empty cells
    column_names = "".join([str(col) for col in pd.read_excel(raw_data_file_path, sheet_name=sheet, header=header_row).columns])
    # If a column name came from an empty cell, it would have "Unnamed" in it
    if "Unnamed" in column_names:
        raise ValueError("""Not all the cells in the chosen row are filled in.
                         Please choose a row that has the name of the columns
                         
                         The values in this row are: {}""".format(column_names))
    else:
        sheet_name_to_everything[sheet]["header_row"] = header_row


Current Sheet Name: CAGE1
Columns Names: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes'], dtype='object')
First few rows of this dataframe:
    runner                 date     match  winner  loser  \
0  Jocelyn  2022-09-06 00:00:00  1.1vs1.2     1.1    1.2   
1      NaN                  NaN  1.2vs1.3     1.2    1.3   
2      NaN                  NaN  1.3vs1.4     1.4    1.3   
3      NaN                  NaN  1.1vs1.3     1.1    1.3   
4      NaN                  NaN  1.2vs1.4     1.2    1.4   

                                               notes  
0                                                NaN  
1                                                NaN  
2                                                NaN  
3  did not realize mouse 3 had 3 matches in a row...  
4                                                NaN  
Type the row number to be used as the header
        (AKA the row with the column name that you want to use.)
        If you want to keep the column names

In [29]:
sheet_name_to_everything

defaultdict(dict,
            {'CAGE1': {'header_row': 0},
             'CAGE2': {'header_row': 0},
             'CAGE3': {'header_row': 0},
             'CAGE4': {'header_row': 0},
             'CAGE5': {'header_row': 0},
             'CAGE6': {'header_row': 0}})

## Reading in all the spreadsheets

In [30]:
# Going through each sheet and creating a dataframe of it
for key, value in sheet_name_to_everything.items():
    value["original_behavior_recording_dataframe"] = pd.read_excel(raw_data_file_path, sheet_name=key, header=value["header_row"])

In [31]:
value["original_behavior_recording_dataframe"].head()

,runner,date,match,winner,loser,notes
0,Jocelyn,2022-09-06 00:00:00,6.1vs6.2,6.1,6.2,NaN
1,NaN,NaN,6.2vs6.3,6.2,6.3,NaN
2,NaN,NaN,6.3vs6.4,6.3,6.4,NaN
3,NaN,NaN,6.4vs6.2,6.2,6.4,NaN
4,NaN,NaN,6.1vs6.3,6.3,6.1,NaN


In [32]:
value["original_behavior_recording_dataframe"].tail()

,runner,date,match,winner,loser,notes
79,NaN,NaN,6.4vs6.1,NaN,NaN,NaN
80,NaN,NaN,6.1vs6.3,NaN,NaN,NaN
81,NaN,NaN,6.2vs6.4,NaN,NaN,NaN
82,NaN,NaN,6.4vs6.3,NaN,NaN,NaN
83,NaN,NaN,6.2vs6.1,NaN,NaN,NaN


## Standarizing the Columns

- Making all the column names lower case and removing any extra spaces in the beginning and at the end
    - One dictionary per sheet

In [33]:
for key, value in sheet_name_to_everything.items():
    # Creating a dictionary that maps the original column name to the standarized one
    column_name_to_standarized = defaultdict(dict)
    for col in value["original_behavior_recording_dataframe"]:
        # Making the column name lower case and removing the spaces
        column_name_to_standarized[col] = "_".join(str(col).lower().strip().split(" "))
    value["column_name_to_standarized"] = column_name_to_standarized

In [34]:
value["column_name_to_standarized"]

defaultdict(dict,
            {'runner': 'runner',
             'date': 'date',
             'match': 'match',
             'winner': 'winner',
             'loser': 'loser',
             'notes': 'notes'})

In [35]:
# Renaming all the columns to the lower case and space removed version
for key, value in sheet_name_to_everything.items():
    value["processed_behavior_recording_dataframe"] = value["original_behavior_recording_dataframe"].rename(columns=value["column_name_to_standarized"])
    value["processed_behavior_recording_dataframe"]["sheet_name"] = key

In [36]:
value["processed_behavior_recording_dataframe"].head()

,runner,date,match,winner,loser,notes,sheet_name
0,Jocelyn,2022-09-06 00:00:00,6.1vs6.2,6.1,6.2,NaN,CAGE6
1,NaN,NaN,6.2vs6.3,6.2,6.3,NaN,CAGE6
2,NaN,NaN,6.3vs6.4,6.3,6.4,NaN,CAGE6
3,NaN,NaN,6.4vs6.2,6.2,6.4,NaN,CAGE6
4,NaN,NaN,6.1vs6.3,6.3,6.1,NaN,CAGE6


## Adding the session number

- We are adding the session number to all the trials. The session number is counting the number of recording sessions that have happened up until that trial. Usually, each session in the spreadsheet is divided up by a session's first row having the date filled in. So we will label a new session when a date is filled in.

# NOTE: If there is a set column that divides the rows up by session, then enter it in the cell below between the quotation marks. Default is `"date"`

In [37]:
all_session_divider_column = ""

In [38]:
for key, value in sheet_name_to_everything.items():
    if not all_session_divider_column:
        # Allowing the user to choose the column that indicates a new session
        session_divider_column = input("""Type the name of the column to be used to divide the rows into sessions. 
        Pick a column that has a value in the cell only with the first row of each session.
        This is usually the "date" column.

        If left blank, the default session divider column will be used. The default is "date"

        The available columns are: {}

        NOTE: If left blank, the column with "date" in the name will be used as the column
        """.format(value["processed_behavior_recording_dataframe"].columns))
    else:
        session_divider_column = all_session_divider_column
        
    # Making the column name lowercase and removing any punctuation
    value["session_divider_column"] = session_divider_column.lower().strip('"').strip("'").strip()

    # Looks for columns with "date" in them if none is inputted
    if not value["session_divider_column"]:
        value["session_divider_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "date" in col.lower()][0]

    # Checks if that column is in the dataframe
    if value["session_divider_column"] not in value["processed_behavior_recording_dataframe"].columns:
        print("WARNING: {} is not a column in {}".format(value["session_divider_column"], key))
        warnings.warn("Look at warning from above or below")
        value["session_divider_column"] = None

Type the name of the column to be used to divide the rows into sessions. 
        Pick a column that has a value in the cell only with the first row of each session.
        This is usually the "date" column.

        If left blank, the default session divider column will be used. The default is "date"

        The available columns are: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes', 'sheet_name'], dtype='object')

        NOTE: If left blank, the column with "date" in the name will be used as the column
        
Type the name of the column to be used to divide the rows into sessions. 
        Pick a column that has a value in the cell only with the first row of each session.
        This is usually the "date" column.

        If left blank, the default session divider column will be used. The default is "date"

        The available columns are: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes', 'sheet_name'], dtype='object')

        NOTE: If left blank, the

In [39]:
value["session_divider_column"]

'date'

In [40]:
for key, value in sheet_name_to_everything.items():
    current_processed_behavior_recording_dataframe = value["processed_behavior_recording_dataframe"].copy()
    try:
        # Getting the indexes of all the rows that have the date or the column that divides the sessions up
        value["session_indexes"] = value["processed_behavior_recording_dataframe"].dropna(axis=0, subset=value["session_divider_column"]).index
        # Adding the session number to each row
        value["processed_behavior_recording_dataframe"] = dataframe.add_session_number_column(value["processed_behavior_recording_dataframe"], value["session_indexes"], session_number_column='session_number')

        # Filling in the NaNs for the session number with the session number from the previous row
        value["processed_behavior_recording_dataframe"]["session_number"] = value["processed_behavior_recording_dataframe"]["session_number"].fillna(method='ffill')
    except:
        print("WARNING: {} does not have a session divider column".format(key))
        warnings.warn("Look at warning from above or below")
        value["processed_behavior_recording_dataframe"] = current_processed_behavior_recording_dataframe
        value["session_indexes"] = None

In [41]:
value["processed_behavior_recording_dataframe"].head()

,runner,date,match,winner,loser,notes,sheet_name,session_number
0,Jocelyn,2022-09-06 00:00:00,6.1vs6.2,6.1,6.2,NaN,CAGE6,1.0
1,NaN,NaN,6.2vs6.3,6.2,6.3,NaN,CAGE6,1.0
2,NaN,NaN,6.3vs6.4,6.3,6.4,NaN,CAGE6,1.0
3,NaN,NaN,6.4vs6.2,6.2,6.4,NaN,CAGE6,1.0
4,NaN,NaN,6.1vs6.3,6.3,6.1,NaN,CAGE6,1.0


In [42]:
value["processed_behavior_recording_dataframe"].tail()

,runner,date,match,winner,loser,notes,sheet_name,session_number
79,NaN,NaN,6.4vs6.1,NaN,NaN,NaN,CAGE6,9.0
80,NaN,NaN,6.1vs6.3,NaN,NaN,NaN,CAGE6,9.0
81,NaN,NaN,6.2vs6.4,NaN,NaN,NaN,CAGE6,9.0
82,NaN,NaN,6.4vs6.3,NaN,NaN,NaN,CAGE6,9.0
83,NaN,NaN,6.2vs6.1,NaN,NaN,NaN,CAGE6,9.0


## Dropping All Rows without a Winner

- Because we will be filling all empty cells with the previous value, we must remove all rows that don't have any data. This is usually cells that don't have any winners or losers filled in the row.
    

# **NOTE**: If there is a set column that is used for the winner and loser,  the rows up by session, then enter it in the cell below between the quotation marks. Default is `"winner"` and `"loser"`

In [43]:
all_winner_id_column = winner_id_column = ""
all_loser_id_column = loser_id_column = ""

In [44]:
for key, value in sheet_name_to_everything.items():
    if not all_winner_id_column:
        # Asking users to specify which column is the one with the winner's information
        value["winner_id_column"] = input("""Type the name of the column of the subject that has won the interaction.
        i.e. "winner"

        The available columns are: {}

        Note: If left blank, the column with "winner" in the name will be used as the column
        """.format(value["processed_behavior_recording_dataframe"].columns))
    
    
        # Looks for columns with "winner" in them if none of inputted
        if not value["winner_id_column"]:
            value["winner_id_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "winner" in col.lower()][0]
    else:
        value["winner_id_column"] = all_winner_id_column
        
    if not all_loser_id_column:   
        # Asking users to specify which column is the one with the winner's information
        value["loser_id_column"] = input("""Type the name of the column of the subject that has won the interaction.
        i.e. "loser"

        The available columns are: {}

        Note: If left blank, the column with "loser" in the name will be used as the column
        """.format(value["processed_behavior_recording_dataframe"].columns))

        # Looks for columns with "loser" in them if none of inputted
        if not value["loser_id_column"]:
            value["loser_id_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "loser" in col.lower()][0]
    else:
        value["loser_id_column"] = all_loser_id_column
    # Dropping all rows that don't have any values in the winner column
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].dropna(subset=value["winner_id_column"])
    # Getting all the floats from the strings, removing any spaces and other characters
    value["processed_behavior_recording_dataframe"][value["winner_id_column"]] = value["processed_behavior_recording_dataframe"][value["winner_id_column"]].astype(str).apply(lambda x: re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)[0] if re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x) else x)
    value["processed_behavior_recording_dataframe"][value["loser_id_column"]] = value["processed_behavior_recording_dataframe"][value["loser_id_column"]].astype(str).apply(lambda x: re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)[0] if re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x) else x)

Type the name of the column of the subject that has won the interaction.
        i.e. "winner"

        The available columns are: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes', 'sheet_name',
       'session_number'],
      dtype='object')

        Note: If left blank, the column with "winner" in the name will be used as the column
        
Type the name of the column of the subject that has won the interaction.
        i.e. "loser"

        The available columns are: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes', 'sheet_name',
       'session_number'],
      dtype='object')

        Note: If left blank, the column with "loser" in the name will be used as the column
        
Type the name of the column of the subject that has won the interaction.
        i.e. "winner"

        The available columns are: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes', 'sheet_name',
       'session_number'],
      dtype='object')

        Note: If left blank, t

In [45]:
value["processed_behavior_recording_dataframe"].head()

,runner,date,match,winner,loser,notes,sheet_name,session_number
0,Jocelyn,2022-09-06 00:00:00,6.1vs6.2,6.1,6.2,NaN,CAGE6,1.0
1,NaN,NaN,6.2vs6.3,6.2,6.3,NaN,CAGE6,1.0
2,NaN,NaN,6.3vs6.4,6.3,6.4,NaN,CAGE6,1.0
3,NaN,NaN,6.4vs6.2,6.2,6.4,NaN,CAGE6,1.0
4,NaN,NaN,6.1vs6.3,6.3,6.1,NaN,CAGE6,1.0


In [46]:
value["processed_behavior_recording_dataframe"].tail()

,runner,date,match,winner,loser,notes,sheet_name,session_number
43,NaN,NaN,6.2vs6.1,6.1,6.2,NaN,CAGE6,8.0
44,NaN,NaN,6.3vs6.2,6.2,6.3,NaN,CAGE6,8.0
45,NaN,NaN,6.1vs6.4,6.1,6.4,NaN,CAGE6,8.0
46,NaN,NaN,6.4vs6.2,6.2,6.4,NaN,CAGE6,8.0
47,NaN,NaN,6.3vs6.1,6.1,6.3,NaN,CAGE6,8.0


# Finding the rows with ties

- If a trial is a tie, there should be another column that indicates that it is a tie. The points will be counted for less. If there is no column, then none of the trials will be counted as ties.

# NOTE: If there is a set column that denotes whether the match has a winner or not, then replace the `None` with the name of the column with quotation marks

In [47]:
all_ties_column = None

In [48]:
for key, value in sheet_name_to_everything.items():
    
    if all_ties_column is None:
        value["ties_column"] = None
    elif all_ties_column:
        value["ties_column"] = all_ties_column
    else:
    
        # Asking users to specify which column is the one with the winner's information
        value["ties_column"] = input("""Type the name of the column that specifies whether or not a tie has occured
        i.e. "ties"

        The available columns are: {}

        Note: If left blank, the column with "ties" in the name will be used as the column. 
        If there are none, then this part will be skipped

        """.format(value["processed_behavior_recording_dataframe"].columns))

        # Looks for columns with "ties" in them if none of inputted
        if not value["ties_column"]:
            if [col for col in value["processed_behavior_recording_dataframe"].columns if "ties" in col.lower()]:
                value["ties_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "ties" in col.lower()][0]
            else: 
                value["ties_column"] = None

# Getting the Session number differences

- Getting the indexes of where each new session starts. So that we can add the session number to each row.

In [49]:
# Example of the columns that will be kept, removing the ties column
list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["ties_column"]]))

['session_number',
 'winner',
 'loser',
 'runner',
 'notes',
 'match',
 'date',
 'sheet_name']

In [50]:
for key, value in sheet_name_to_everything.items():
    # Removing tie columns because not all rows are ties, so we do not want to fill them in
    if value["ties_column"]:
        non_ties_columns = list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["ties_column"]]))
        value["processed_behavior_recording_dataframe"][non_ties_columns] = value["processed_behavior_recording_dataframe"][non_ties_columns].fillna(method='ffill')
    else:
        # Filling all the empty cells with the value in the previous cell
        value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].fillna(method='ffill')

    try:
        # Seeing which rows have a different session than the previous one
        # This will be used to plot vertical lines for each new session
        value["processed_behavior_recording_dataframe"]['session_number_difference'] = value["processed_behavior_recording_dataframe"]['session_number'].diff()
    except:
        print("WARNING: {} does not have a session divider column".format(key))
        warnings.warn("Look at warning from above or below")

In [51]:
value["processed_behavior_recording_dataframe"].head(n=15)

,runner,date,match,winner,loser,notes,sheet_name,session_number,session_number_difference
0,Jocelyn,2022-09-06,6.1vs6.2,6.1,6.2,NaN,CAGE6,1.0,NaN
1,Jocelyn,2022-09-06,6.2vs6.3,6.2,6.3,NaN,CAGE6,1.0,0.0
2,Jocelyn,2022-09-06,6.3vs6.4,6.3,6.4,NaN,CAGE6,1.0,0.0
3,Jocelyn,2022-09-06,6.4vs6.2,6.2,6.4,NaN,CAGE6,1.0,0.0
4,Jocelyn,2022-09-06,6.1vs6.3,6.3,6.1,NaN,CAGE6,1.0,0.0
5,Jocelyn,2022-09-06,6.4vs6.1,6.1,6.4,NaN,CAGE6,1.0,0.0
6,Meghan,2022-09-07,6.3vs6.2,6.3,6.2,NaN,CAGE6,2.0,1.0
7,Meghan,2022-09-07,6.4vs6.1,6.1,6.4,NaN,CAGE6,2.0,0.0
8,Meghan,2022-09-07,6.2vs6.4,6.2,6.4,NaN,CAGE6,2.0,0.0
9,Meghan,2022-09-07,6.1vs6.3,6.1,6.3,NaN,CAGE6,2.0,0.0


In [52]:
value["processed_behavior_recording_dataframe"].tail(n=15)

,runner,date,match,winner,loser,notes,sheet_name,session_number,session_number_difference
33,Jocelyn,2022-09-13,6.4vs6.2,6.2,6.4,NaN,CAGE6,6.0,0.0
34,Jocelyn,2022-09-13,6.1vs6.3,6.1,6.3,NaN,CAGE6,6.0,0.0
35,Jocelyn,2022-09-13,6.4vs6.1,6.1,6.4,NaN,CAGE6,6.0,0.0
36,Meghan,2022-09-15,6.3vs6.2,6.3,6.2,NaN,CAGE6,7.0,1.0
37,Meghan,2022-09-15,6.4vs6.1,6.1,6.4,NaN,CAGE6,7.0,0.0
38,Meghan,2022-09-15,6.2vs6.4,6.2,6.4,NaN,CAGE6,7.0,0.0
39,Meghan,2022-09-15,6.1vs6.3,6.1,6.3,NaN,CAGE6,7.0,0.0
40,Meghan,2022-09-15,6.3vs6.4,6.3,6.4,NaN,CAGE6,7.0,0.0
41,Meghan,2022-09-15,6.1vs6.2,6.2,6.1,NaN,CAGE6,7.0,0.0
42,Meghan,2022-09-27,6.4vs6.3,6.3,6.4,NaN,CAGE6,8.0,1.0


# TODO: EDIT START HERE Add cage informationhere

## Getting the cage number

- The cage number is sometimes not consistent throughout a spreadsheet for the same cage. So we will try to standarize it into one value.
    - **NOTE**: This assumes cage numbers are actual numbers. And not entirely consisting of letters. If that isn't the case, then you must edit this cell for your needs.

# NOTE: If there is a column that has the cage number information, replace the `None` with the column name in quotation marks

- Change this when you have winner and loser cage

In [53]:
cage_num_of_winner_column = None
cage_num_of_loser_column = None

- Change this when you have id to cage

In [54]:
id_to_cage = {}

In [55]:
for key, value in sheet_name_to_everything.items():   
    # When you have a dictionary of IDs to Cage Numbers
    if id_to_cage:
        # Specifying the name by default
        value["cage_num_of_winner_column"] = "original_cage_num_of_winner"
        value["cage_num_of_loser_column"] = "original_cage_num_of_loser"
        # Mapping the IDs to the cage number
        value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]] = value["processed_behavior_recording_dataframe"][value["winner_id_column"]].map(id_to_cage)
        value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]] = value["processed_behavior_recording_dataframe"][value["loser_id_column"]].map(id_to_cage)
        continue
        
    # When you have a column that specifies the cage number
    elif cage_num_of_winner_column and cage_num_of_loser_column:
        value["cage_num_of_winner_column"] = cage_num_of_winner_column
        value["cage_num_of_loser_column"] = cage_num_of_loser_column
    elif cage_num_of_winner_column:
        value["cage_num_of_winner_column"] = cage_num_of_winner_column
    elif cage_num_of_loser_column:
        value["cage_num_of_loser_column"] = cage_num_of_loser_column

    
    if not cage_num_of_winner_column:
        # Asking the user which column has the cage number
        value["cage_num_of_winner_column"] = input("""Type the name of the column of the cage of the WINNER subject
        i.e. "cage_num_of_winner"

        The available columns are: {}

        Note: If left blank, the column with "winner" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        """.format(value["processed_behavior_recording_dataframe"].columns))        
        
        # Getting the column that has "winner" and "cage" in its name if no column is specified
        if not value["cage_num_of_winner_column"]:
            # Checking to see if there are any columns with the winner and cage in the name
            column_names_with_cage = [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower() and "winner" in col.lower()]
            # Using the column with cage in the name if there are none with winner and cage
            column_names_with_cage += [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower()]
            if column_names_with_cage:
                value["winnner_cage_column"] = column_names_with_cage[0]
            else:
                # Using the default name as the cage column name if there are none
                value["cage_num_of_winner_column"] = "original_cage_num_of_winner"
    
    if not cage_num_of_loser_column:
        # Asking the user which column has the cage number
        value["cage_num_of_loser_column"] = input("""Type the name of the column of the cage of the LOSER subject
        i.e. "cage_num_of_loser"

        The available columns are: {}

        Note: If left blank, the column with "loser" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        """.format(value["processed_behavior_recording_dataframe"].columns))        
        
        # Getting the column that has "loser" and "cage" in its name if no column is specified
        if not value["cage_num_of_loser_column"]:
            # Checking to see if there are any columns with the loser and cage in the name
            column_names_with_cage = [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower() and "loser" in col.lower()]
            # Using the column with cage in the name if there are none with loser and cage
            column_names_with_cage += [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower()]
            if column_names_with_cage:
                value["winnner_cage_column"] = column_names_with_cage[0]
            else:
                # Using the default name as the cage column name if there are none
                value["cage_num_of_loser_column"] = "original_cage_num_of_loser"
    
    # Using the spreadsheet name as the cage number if there are no columns that match
    if value["cage_num_of_winner_column"] not in value["processed_behavior_recording_dataframe"].columns:
        value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]] = key

    if value["cage_num_of_loser_column"] not in value["processed_behavior_recording_dataframe"].columns:
        value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]] = key       
    
    # Turning the cage names into the float number only
    # Or using the same name if there are no floats
    value["processed_behavior_recording_dataframe"]["processed_cage_num_of_winner"] = value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]].apply(lambda x: re.findall(r'\d+', str(x))[0] if re.findall(r'\d+', str(x)) else x)
    value["processed_behavior_recording_dataframe"]["processed_cage_num_of_loser"] = value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]].apply(lambda x: re.findall(r'\d+', str(x))[0] if re.findall(r'\d+', str(x)) else x)

    

Type the name of the column of the cage of the WINNER subject
        i.e. "cage_num_of_winner"

        The available columns are: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes', 'sheet_name',
       'session_number', 'session_number_difference'],
      dtype='object')

        Note: If left blank, the column with "winner" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        
Type the name of the column of the cage of the LOSER subject
        i.e. "cage_num_of_loser"

        The available columns are: Index(['runner', 'date', 'match', 'winner', 'loser', 'notes', 'sheet_name',
       'session_number', 'session_number_difference'],
      dtype='object')

        Note: If left blank, the column with "loser" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        
Type the name of the column of the cage of the WINNER subject
        i.e. "cage_num_of_winner"

        The available colu

In [56]:
sheet_name_to_everything["CAGE1"]["processed_behavior_recording_dataframe"]

,runner,date,match,winner,loser,notes,sheet_name,session_number,session_number_difference,original_cage_num_of_winner,original_cage_num_of_loser,processed_cage_num_of_winner,processed_cage_num_of_loser
0,Jocelyn,2022-09-06,1.1vs1.2,1.1,1.2,NaN,CAGE1,1.0,NaN,CAGE1,CAGE1,1,1
1,Jocelyn,2022-09-06,1.2vs1.3,1.2,1.3,NaN,CAGE1,1.0,0.0,CAGE1,CAGE1,1,1
2,Jocelyn,2022-09-06,1.3vs1.4,1.4,1.3,NaN,CAGE1,1.0,0.0,CAGE1,CAGE1,1,1
3,Jocelyn,2022-09-06,1.1vs1.3,1.1,1.3,did not realize mouse 3 had 3 matches in a row...,CAGE1,1.0,0.0,CAGE1,CAGE1,1,1
4,Jocelyn,2022-09-06,1.2vs1.4,1.2,1.4,did not realize mouse 3 had 3 matches in a row...,CAGE1,1.0,0.0,CAGE1,CAGE1,1,1
5,Jocelyn,2022-09-06,1.4vs1.1,1.1,1.4,did not realize mouse 3 had 3 matches in a row...,CAGE1,1.0,0.0,CAGE1,CAGE1,1,1
6,Meghan,2022-09-07,1.3vs1.2,1.2,1.3,did not realize mouse 3 had 3 matches in a row...,CAGE1,2.0,1.0,CAGE1,CAGE1,1,1
7,Meghan,2022-09-07,1.4vs1.1,1.1,1.4,did not realize mouse 3 had 3 matches in a row...,CAGE1,2.0,0.0,CAGE1,CAGE1,1,1
8,Meghan,2022-09-07,1.2vs1.4,1.2,1.4,did not realize mouse 3 had 3 matches in a row...,CAGE1,2.0,0.0,CAGE1,CAGE1,1,1
9,Meghan,2022-09-07,1.1vs1.3,1.1,1.3,did not realize mouse 3 had 3 matches in a row...,CAGE1,2.0,0.0,CAGE1,CAGE1,1,1


## Calculating Elo rating

- Example calculation

In [57]:
calculation.calculate_elo_rating(subject_elo_rating=1000, agent_elo_rating=2000, score=1)

1019.9

In [58]:
calculation.update_elo_rating(winner_id="A", loser_id="B")

defaultdict(<function elorating.calculation.update_elo_rating.<locals>.<lambda>()>,
            {'A': 1010.0, 'B': 990.0})

## Get the Elo rating for all the events

- Going through each row or interaction and calculating the new Elo rating for the winner and loser. This will create a new dataframe based off of the processed behavioral recording dataframe

# NOTE: If there are a set of columns to keep, edit the cell below with the name of the columns each in quotation marks seperated by commas
   - i.e. `['runner', 'date', 'match', 'winner', 'loser', 'notes', 'session_number',
       'session_number_difference']`

# TODO: Edit start

In [59]:
all_columns_to_keep = [""]

In [ ]:
def get_subject_cage_number(x):
    """
    """
    if x["win_draw_loss"] == 1:
        return x["processed_cage_num_of_winner"]
    
    x: x["processed_cage_num_of_winner"] if x["win_draw_loss"] == 1 else (x["processed_cage_num_of_loser"] if x["win_draw_loss"] == 0 else x["processed_cage_num_of_winner"]), axis = 1)


In [60]:
for key, value in sheet_name_to_everything.items():
    if not all_columns_to_keep:
        # Asking the user which columns to carry over to the Elo rating calculation dataframe
        value["columns_to_keep_string"] = input("""Type all the columns that will be copied from the original dataframe to the Elo rating calculated dataframe. 

        All the available actions: {}
        Each column must be put in quotes and seperated by a comma(,). i.e. 'length of observations', 'date', 'cage #'

        NOTE: If left blank, all the columns will be kept
        """.format(value["processed_behavior_recording_dataframe"].columns))


        # Making a list out of the string of inputted sheet names
        if value["columns_to_keep_string"]:
            value["columns_to_keep_string"] = "[" + value["columns_to_keep_string"] + "]"
            value["columns_to_keep_list"] = ast.literal_eval(value["columns_to_keep_string"])
    else: 
        value["columns_to_keep_list"] = all_columns_to_keep
    try:
        # Calculating the Elo rating    
        value["index_to_elo_rating_and_meta_data"] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=value["processed_behavior_recording_dataframe"], \
                                                                                                                   winner_id_column=value["winner_id_column"], loser_id_column=value["loser_id_column"], \
                                                                                                                             additional_columns=value["columns_to_keep_list"], tie_column=value["ties_column"])
    
    except:
        nonexistent_columns = set(value["columns_to_keep_list"]) - set(value["processed_behavior_recording_dataframe"].columns)
        print("WARNING: {} does not have {} columns".format(key, nonexistent_columns))
        warnings.warn("Look at warning from above or below")
        # Using all the column names if no column name is specified
        # Removing the winner and loser column because they will be specified as the columns for the Elo rating calculation
        value["columns_to_keep_list"] =  sorted(list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["winner_id_column"]]) - set([value["loser_id_column"]])))
        value["index_to_elo_rating_and_meta_data"] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=value["processed_behavior_recording_dataframe"], \
                                                                                                                   winner_id_column=value["winner_id_column"], loser_id_column=value["loser_id_column"], \
                                                                                                                             additional_columns=value["columns_to_keep_list"], tie_column=value["ties_column"])     

    # Making a dataframe from the dictionary 
    value["elo_rating_dataframe"] = pd.DataFrame.from_dict(value["index_to_elo_rating_and_meta_data"], orient="index")
    value["elo_rating_dataframe"]["cage_num_of_subject"] = value["elo_rating_dataframe"].apply(lambda x: x["processed_cage_num_of_winner"] if x["win_draw_loss"] == 1 else (x["processed_cage_num_of_loser"] if x["win_draw_loss"] == 0 else x["processed_cage_num_of_winner"]), axis = 1)
#     value["elo_rating_dataframe"]["cage_num_of_agent"] = 


C:\Users\Leo\AppData\Local\Temp\ipykernel_17980\206115594.py:28: UserWarning: Look at warning from above or below
  warnings.warn("Look at warning from above or below")
C:\Users\Leo\AppData\Local\Temp\ipykernel_17980\206115594.py:28: UserWarning: Look at warning from above or below
  warnings.warn("Look at warning from above or below")
C:\Users\Leo\AppData\Local\Temp\ipykernel_17980\206115594.py:28: UserWarning: Look at warning from above or below
  warnings.warn("Look at warning from above or below")
C:\Users\Leo\AppData\Local\Temp\ipykernel_17980\206115594.py:28: UserWarning: Look at warning from above or below
  warnings.warn("Look at warning from above or below")
C:\Users\Leo\AppData\Local\Temp\ipykernel_17980\206115594.py:28: UserWarning: Look at warning from above or below
  warnings.warn("Look at warning from above or below")
C:\Users\Leo\AppData\Local\Temp\ipykernel_17980\206115594.py:28: UserWarning: Look at warning from above or below
  warnings.warn("Look at warning from abo

In [61]:
value["elo_rating_dataframe"]

,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,date,...,notes,original_cage_num_of_loser,original_cage_num_of_winner,processed_cage_num_of_loser,processed_cage_num_of_winner,runner,session_number,session_number_difference,sheet_name,cage_num_of_subject
0,1,6.1,6.2,1000.0,1010.0,1,1,2,0,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,NaN,CAGE6,6
1,1,6.2,6.1,1000.0,990.0,0,2,1,1,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,NaN,CAGE6,6
2,2,6.2,6.3,990.0,1000.3,1,2,3,0,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,0.0,CAGE6,6
3,2,6.3,6.2,1000.0,989.7,0,3,2,1,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,0.0,CAGE6,6
4,3,6.3,6.4,989.7,1000.0,1,3,4,0,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,0.0,CAGE6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,46,6.4,6.1,843.1,839.8,0,4,1,1,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6
92,47,6.2,6.4,1015.9,1021.2,1,2,4,0,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6
93,47,6.4,6.2,839.8,834.5,0,4,2,1,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6
94,48,6.1,6.3,1131.0,1137.7,1,1,3,0,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6


In [62]:
value["elo_rating_dataframe"].head()

,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,date,...,notes,original_cage_num_of_loser,original_cage_num_of_winner,processed_cage_num_of_loser,processed_cage_num_of_winner,runner,session_number,session_number_difference,sheet_name,cage_num_of_subject
0,1,6.1,6.2,1000.0,1010.0,1,1,2,0,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,NaN,CAGE6,6
1,1,6.2,6.1,1000.0,990.0,0,2,1,1,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,NaN,CAGE6,6
2,2,6.2,6.3,990.0,1000.3,1,2,3,0,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,0.0,CAGE6,6
3,2,6.3,6.2,1000.0,989.7,0,3,2,1,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,0.0,CAGE6,6
4,3,6.3,6.4,989.7,1000.0,1,3,4,0,2022-09-06,...,NaN,CAGE6,CAGE6,6,6,Jocelyn,1.0,0.0,CAGE6,6


In [63]:
value["elo_rating_dataframe"].tail()

,total_match_number,subject_id,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,date,...,notes,original_cage_num_of_loser,original_cage_num_of_winner,processed_cage_num_of_loser,processed_cage_num_of_winner,runner,session_number,session_number_difference,sheet_name,cage_num_of_subject
91,46,6.4,6.1,843.1,839.8,0,4,1,1,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6
92,47,6.2,6.4,1015.9,1021.2,1,2,4,0,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6
93,47,6.4,6.2,839.8,834.5,0,4,2,1,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6
94,48,6.1,6.3,1131.0,1137.7,1,1,3,0,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6
95,48,6.3,6.1,1013.3,1006.6,0,3,1,1,2022-09-27,...,NaN,CAGE6,CAGE6,6,6,Meghan,8.0,0.0,CAGE6,6


In [64]:
value["elo_rating_dataframe"].groupby("subject_id").count()

,total_match_number,agent_id,original_elo_rating,updated_elo_rating,win_draw_loss,subject_ranking,agent_ranking,pairing_index,date,match,notes,original_cage_num_of_loser,original_cage_num_of_winner,processed_cage_num_of_loser,processed_cage_num_of_winner,runner,session_number,session_number_difference,sheet_name,cage_num_of_subject
subject_id,,,,,,,,,,,,,,,,,,,,
6.1,24,24,24,24,24,24,24,24,24,24,0,24,24,24,24,24,24,23,24,24
6.2,24,24,24,24,24,24,24,24,24,24,0,24,24,24,24,24,24,23,24,24
6.3,24,24,24,24,24,24,24,24,24,24,0,24,24,24,24,24,24,24,24,24
6.4,24,24,24,24,24,24,24,24,24,24,0,24,24,24,24,24,24,24,24,24


In [65]:
raise ValueError()

ValueError: 

# TODO: Refactor starting here

## Combining all the Elo rating dataframes into one

In [ ]:
# Putting all the dataframes into one list
all_sheet_elo_scord_dataframe_list = []
for key, value in sheet_name_to_everything.items():    
    all_sheet_elo_scord_dataframe_list.append(value["elo_rating_dataframe"])

# Combining all the dataframes into one
all_sheet_elo_scord_dataframe_combined = pd.concat(all_sheet_elo_scord_dataframe_list)

In [ ]:
all_sheet_elo_scord_dataframe_combined

- Adding the strain information

In [ ]:
all_subject_ids = set(all_sheet_elo_scord_dataframe_combined["subject_id"].unique()).union(set(all_sheet_elo_scord_dataframe_combined["agent_id"].unique()))

# NOTE: If there are strains that are associated to each cage, then create a dictionary of cage numbers to strains inside the `{}`
- i.e. `cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}`

In [ ]:
cage_to_strain = {}

In [ ]:
if cage_to_strain:
    all_sheet_elo_scord_dataframe_combined["strain"] = all_sheet_elo_scord_dataframe_combined["processed_cage_number"].map(cage_to_strain)

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

- Adding the name of the experiment

In [ ]:
# Adding the name of the experiment
all_sheet_elo_scord_dataframe_combined["experiment_type"] = protocol_name

- Adding the cohort

In [ ]:
all_sheet_elo_scord_dataframe_combined["cohort"] = cohort_name

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

In [ ]:
# Checking to see how many rows for each subject in each cage
all_sheet_elo_scord_dataframe_combined.groupby(['subject_id','processed_cage_num_of_winner']).count()

## Creating a dataframe with only the final Elo rating for each subject

In [ ]:
# Checking to see which cage and subject combination has more than one row
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_combined.groupby(['subject_id','processed_cage_num_of_winner']).size().reset_index()
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_groupby[all_sheet_elo_scord_dataframe_groupby[0] >= 1]

# Going through each combination and saving the combination to a dictionary
index_to_subject_id_and_processed_cage_number = defaultdict(dict)
for index, row in all_sheet_elo_scord_dataframe_groupby.iterrows():
    index_to_subject_id_and_processed_cage_number[index]['subject_id'] = row['subject_id']
    index_to_subject_id_and_processed_cage_number[index]['processed_cage_num_of_winner'] = row['processed_cage_num_of_winner']


In [ ]:
index_to_subject_id_and_processed_cage_number

- Getting the final Elo rating for each cage and subject combination

In [ ]:
for index, (key, value) in enumerate(index_to_subject_id_and_processed_cage_number.items()):   
    # The Elo rating dataframe for each cage and subject combination
    per_subject_dataframe = all_sheet_elo_scord_dataframe_combined[(all_sheet_elo_scord_dataframe_combined["subject_id"] == value["subject_id"]) & (all_sheet_elo_scord_dataframe_combined["processed_cage_num_of_winner"] == value["processed_cage_num_of_winner"])]
    # Getting the final Elo rating for each combination
    # -1 Means that we're getting the data from the last row

    index_to_subject_id_and_processed_cage_number[index]["final_elo_rating"] = per_subject_dataframe.iloc[-1]["updated_elo_rating"]
    index_to_subject_id_and_processed_cage_number[index]["cohort"] = per_subject_dataframe.iloc[-1]["cohort"]
    try:
        index_to_subject_id_and_processed_cage_number[index]["strain"] = per_subject_dataframe.iloc[-1]["strain"]
    except:
        print("WARNING: {} in cage {} does not have strain information".format(index_to_subject_id_and_processed_cage_number[key]["subject_id"], index_to_subject_id_and_processed_cage_number[key]["processed_cage_num_of_winner"]))
        warnings.warn("Look at warning from above or below")


In [ ]:
index_to_subject_id_and_processed_cage_number

In [ ]:
id_to_final_elo_rating_df = pd.DataFrame.from_dict(index_to_subject_id_and_processed_cage_number, orient="index")
# Adding protocol name
id_to_final_elo_rating_df["experiment_type"] = protocol_name
# Adding rank
id_to_final_elo_rating_df["rank"] = id_to_final_elo_rating_df.groupby("processed_cage_num_of_winner")["final_elo_rating"].rank("dense", ascending=False)
# Sorting by cage and then id
id_to_final_elo_rating_df = id_to_final_elo_rating_df.sort_values(by=['processed_cage_num_of_winner', "subject_id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_rating_df.head()

In [ ]:
id_to_final_elo_rating_df.tail()

# TODO: Remove depending on how cage or id to strain is set up

# TODO: Edit done

In [ ]:
raise ValueError()

# Making plots for all sheets

- Getting the dates the files were being recorded to use for the file name

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

- Getting the earliest and the latest dates for all the recordings

In [ ]:
all_earlist_dates = []
all_latest_dates = []
for key, value in sheet_name_to_everything.items():
    # Getting all the earliest dates for each sheet
    all_earlist_dates.append(value["elo_rating_dataframe"][value["session_divider_column"]].min())
    # Getting all the latest dates for each sheet
    all_latest_dates.append(value["elo_rating_dataframe"][value["session_divider_column"]].max())

In [ ]:
all_earlist_dates

In [ ]:
min(all_earlist_dates)

In [ ]:
all_latest_dates

In [ ]:
max(all_latest_dates)

- Turning the Dates into a easier to read format

In [ ]:
earliest_date = str(min(all_earlist_dates)).split()[0]

In [ ]:
latest_date = str(max(all_latest_dates)).split()[0]

In [ ]:
earliest_date

In [ ]:
latest_date

- Getting the cage numbers

In [ ]:
all_cages_list = []
# Creating a list of all the cage numbers
for key, value in sheet_name_to_everything.items():
    for cage in value["elo_rating_dataframe"]["processed_cage_number"].unique():
        all_cages_list.append(cage)


In [ ]:
all_cages_string = "_".join(sorted([sheet.lower().strip("cage").strip() for sheet in all_cages_list]))

In [ ]:
all_cages_string

- Creating an output directory to save the plots

In [ ]:
plot_output_directory = os.path.join(".", "proc", "plots", "{}_elo_rating".format(protocol_name))

In [ ]:
plot_output_directory

In [ ]:
os.makedirs(plot_output_directory, exist_ok=True)

# **NOTE**: Sometimes this cell needs to be run again to make sure the size is correct

In [ ]:
plt.rcParams["figure.figsize"] = (13.5,7.5)

In [ ]:
# Getting the highest and lowest Elo rating for cutoffs of the Y-axis
max_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].max()
min_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].min()

# Making a plot for each sheet
for key, value in sheet_name_to_everything.items():
    # Setting the size of the figure
    plt.rcParams["figure.figsize"] = (13.5,7.5)
    print(key)
    elo_rating_dataframe = value["elo_rating_dataframe"]
    # Using a new figure template for each sheet
    fig, ax = plt.subplots()        
        
    # Drawing vertical lines that represent when each session begins
    # Based on when a row has a different session than the previous row
    for index, row in elo_rating_dataframe[elo_rating_dataframe['session_number_difference'] != 0.0].iterrows():
        # Offsetting by 0.5 to avoid drawing the line on the dot
        # Drawing the lines a little above the max and a little below the minimum
        plt.vlines(x=[row["total_match_number"] - 0.5], ymin=min_elo_rating-50, ymax=max_elo_rating+50, colors='black', linestyle='dashed')

    # Drawing a line for each subject
    for subject in sorted(elo_rating_dataframe["subject_id"].unique()):
        # Getting all the rows with the current subject
        subject_dataframe = elo_rating_dataframe[elo_rating_dataframe["subject_id"] == subject]
        # Making the current match number the X-Axis
        plt.plot(subject_dataframe["total_match_number"], subject_dataframe["updated_elo_rating"], '-o', label=subject)

    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo rating")
    # Formattnig Cohort and Experiment Name so that it's more readable with spacing and capitalization
    ax.set_title("{} Elo rating for {}: {}".format(" ".join(cohort_name.split("_")).capitalize(), string.capwords(" ".join(protocol_name.split("_"))), key))
    
    # To show the legend
    ax.legend(loc="upper left")
    # Setting the values of the Y-axis
    plt.ylim(min_elo_rating-50, max_elo_rating+50) 
    # Saving the plot
    file_name = "{}_{}_elo_rating_{}_date_{}_{}.png".format(cohort_name, prefix_name, key, earliest_date, latest_date)
    # Removing all the spaces and replacing them with underscores
    file_name = "_".join(file_name.split(" "))
    plt.savefig(os.path.join(plot_output_directory, file_name))
    # Showing the plots
    plt.show()

# Saving the Dataframes

- Creating a subfolder to put the Elo rating Spreadsheets

In [ ]:
elo_rating_spreadsheet_output_directory = os.path.join(".", "proc", "elo_rating_spread_sheets", "{}".format(protocol_name))

In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
os.makedirs(elo_rating_spreadsheet_output_directory, exist_ok=True)

- Saving the dataframes to a file

In [ ]:
file_name = "{}_{}_elo_rating_history_cages_{}_date_{}_{}.csv".format(cohort_name, prefix_name, all_cages_string, earliest_date, latest_date)
all_sheet_elo_scord_dataframe_combined.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))

In [ ]:
file_name = "{}_{}_final_elo_rating_cages_{}_date_{}_{}.csv".format(cohort_name, prefix_name, all_cages_string, earliest_date, latest_date)
id_to_final_elo_rating_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))

# Seeing which subject is the dominant or submissive

- Grouping all the rows with the same pair

In [ ]:
all_processed_behavior_recording_list = []
for key, value in sheet_name_to_everything.items():
    all_processed_behavior_recording_list.append(value["processed_behavior_recording_dataframe"])
    

- Combining all the dataframes from all the cages

In [ ]:
all_processed_behavior_recording_df = pd.concat(all_processed_behavior_recording_list)

In [ ]:
all_processed_behavior_recording_df.head()

- Getting a tuple of the animal IDs to be able to group

In [ ]:
# Getting the animal IDs from the Match string
# i.e. Getting all the floats and removing all spaces
# Sorting so that the animal IDs are always in the same order
all_processed_behavior_recording_df["animal_id"] =  all_processed_behavior_recording_df["match"].apply(lambda x: sorted([subject_id.lower().strip() for subject_id in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)]))


In [ ]:
# Making a tuple out of the list
# Tuples are used because lists are mutable and can't be grouped with
all_processed_behavior_recording_df["tuple_animal_id"] = all_processed_behavior_recording_df["animal_id"].apply(lambda x: tuple(x))

In [ ]:
all_processed_behavior_recording_df.head()

- Removing columns that would be unnecessary for the pairings

In [ ]:
all_processed_behavior_recording_df.columns

In [ ]:
# Getting only the columns that we need
all_processed_behavior_recording_df = all_processed_behavior_recording_df[['runner', 'date', 'match', 'winner', 'loser', 'notes', 'animal_id', 'tuple_animal_id', "processed_cage_number"]]
# Converting the cage number to an integer
all_processed_behavior_recording_df['processed_cage_number'] = all_processed_behavior_recording_df['processed_cage_number'].astype(int)

In [ ]:
all_processed_behavior_recording_df.head()

- Getting the ID of the winner and the loser for each pair with each match

In [ ]:
all_wins_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['winner'].apply(list)
all_loses_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['loser'].apply(list)

In [ ]:
all_wins_per_pair[:5]

- Making a dataframe of all the winner IDs and all the loser IDs for a given pair

In [ ]:
all_competition_per_pair_df = pd.concat([all_wins_per_pair, all_loses_per_pair], axis=1).reset_index()

In [ ]:
all_competition_per_pair_df = all_competition_per_pair_df.rename(columns={k: prefix_name + "_" + k for k in all_competition_per_pair_df.columns})

In [ ]:
all_competition_per_pair_df

- Adding the cage information

In [ ]:
# Getting the cage number for each pair
dropped_duplicate_all_processed_behavior_recording_df = all_processed_behavior_recording_df[["tuple_animal_id", "processed_cage_number"]].drop_duplicates()

In [ ]:
dropped_duplicate_all_processed_behavior_recording_df.head()

- Creating a dictionary so that we can create a column for the cage number based on the IDs

## TODO: EDIT below

In [ ]:
try:
    pair_to_cage = pd.Series(dropped_duplicate_all_processed_behavior_recording_df["processed_cage_number"].values, index=dropped_duplicate_all_processed_behavior_recording_df["tuple_animal_id"]).to_dict()
    display(pair_to_cage)
except:
    #TODO: 
    pass

In [ ]:
try:
    all_competition_per_pair_df["processed_cage_number"] = all_competition_per_pair_df["{}_tuple_animal_id".format(prefix_name)].map(pair_to_cage)
    all_competition_per_pair_df["processed_cage_number"] = all_competition_per_pair_df["processed_cage_number"].astype(int).astype(str)
    display(all_competition_per_pair_df.head())
except:
    pass

In [ ]:
all_competition_per_pair_df["cohort"] = cohort_name


- Calculating the overall winner and loser. Also seeing if there is signficant difference in the number of wins to see if one is dominant over the other

In [ ]:
all_competition_per_pair_df["{}_averaged_winner".format(prefix_name)] = all_competition_per_pair_df["{}_winner".format(prefix_name)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_averaged_loser".format(prefix_name)] = all_competition_per_pair_df["{}_loser".format(prefix_name)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_winner_count".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(prefix_name)].count(x["{}_averaged_winner".format(prefix_name)]), axis=1)
all_competition_per_pair_df["{}_loser_count".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(prefix_name)].count(x["{}_averaged_loser".format(prefix_name)]), axis=1)
all_competition_per_pair_df["{}_count_difference".format(prefix_name)] = all_competition_per_pair_df["{}_winner_count".format(prefix_name)] - all_competition_per_pair_df["{}_loser_count".format(prefix_name)]
all_competition_per_pair_df["{}_match_count".format(prefix_name)] = all_competition_per_pair_df["{}_winner".format(prefix_name)].apply(lambda x: len(x))
all_competition_per_pair_df["{}_percent_win".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner_count".format(prefix_name)] / x["{}_match_count".format(prefix_name)], axis=1)
all_competition_per_pair_df["{}_percentage_tie".format(prefix_name)] = all_competition_per_pair_df["{}_percent_win".format(prefix_name)].apply(lambda x: True if x < 0.75 else False)

In [ ]:
all_competition_per_pair_df

- Saving the competiton pair results dataframe to a file

In [ ]:
file_name = "{}_{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(cohort_name, prefix_name, all_cages_string, earliest_date, latest_date)


In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:



all_competition_per_pair_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))